# Try out first CNN on EEG data with labels

## Pre-processing
+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.

## Train CNN network
+ Define network architecture
+ Split data
+ Train model

### Use mcfly for some first model testing: https://github.com/NLeSC/mcfly

## Import packages & links

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mne
#%matplotlib inline

from mayavi import mlab

In [2]:
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_ePodium\\"
PATH_CODE = ROOT + "EEG_explorer\\"
PATH_DATA = ROOT + "Data\\"
PATH_OUTPUT = ROOT + "Data\\processed\\"
PATH_METADATA = ROOT + "Data\\metadata\\"
file_labels = "metadata.xlsx"

import sys
sys.path.insert(0, PATH_CODE)

## Load pre-processed dataset
+ See notebook for preprocessing: Exploring_EEG_data_04_prepare_data_for_ML.ipynb

In [3]:
filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected.npy"
signal_collection = np.load(filename)

filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected_labels.npy"
label_collection = np.load(filename)

## Split training data
The entire dataset is split into:
+ training data (here: about 70%) which is used to train a model.
+ validation data, used to monitor the model progress and avoid overfitting.
+ testing data, meant for final check on model performance.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(signal_collection, label_collection, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [5]:
print('Train set size:', X_train.shape[0])
print('Validation set size:', X_val.shape[0])
print('Test set size:', X_test.shape[0])

Train set size: 28237
Validation set size: 4983
Test set size: 5863


## Switch to 1-hot encoding for labels
+ We have six categories or classes. Those are best represented by a so called 1-hot encoding. This means nothing else than simply a binary 0-or-1 for every class.. 

In [6]:
from sklearn.preprocessing import LabelBinarizer
label_transform = LabelBinarizer()

y_train_binary = label_transform.fit_transform(y_train.astype(int))
y_val_binary = label_transform.fit_transform(y_val.astype(int))
y_test_binary = label_transform.fit_transform(y_test.astype(int))

In [7]:
y_val_binary[:10,:]

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [8]:
# Show found labels:
label_transform.classes_

array([  3,   6,  13,  26,  66, 132])

Check distribution accross the 6 label categories:

In [9]:
labels = list(label_transform.classes_)
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
3,0.313206
6,0.488437
13,0.038885
26,0.061515
66,0.038141
132,0.059815


### Note:
We have more data on group 2 than on group 1. And far more data for stimuli 3 than for stimuli 13 and 66 (not surprising). 

--> post on balancing datasets: https://towardsdatascience.com/handling-imbalanced-datasets-in-deep-learning-f48407a0e758

### Needs some thinking on how to balance the data set !
e.g. by frequency dependend selection rule, or by defining a suitied special loss function....

In [10]:
import mcfly

num_classes = y_train_binary.shape[1]
models = mcfly.modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 8)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
models_to_print = range(len(models))
for i, item in enumerate(models):
    if i in models_to_print:
        model, params, model_types = item
        print("-------------------------------------------------------------------------------------------------------")
        print("Model " + str(i))
        print(" ")
        print("Hyperparameters:")
        print(params)
        print(" ")
        print("Model description:")
        model.summary()
        print(" ")
        print("Model type:")
        print(model_types)
        print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.0001353096649122913, 'regularization_rate': 0.08347912108149602, 'filters': array([ 60,  70, 100,  45,  39,  41,  27,  25]), 'fc_hidden_nodes': 1894}
 
Model description:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 30, 501)           2004      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 60)            90240     
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 60)            240       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 60)            0         
_________________________________________________________________
conv1d_2 (Conv1D) 

In [12]:
outputfile = os.path.join(PATH_OUTPUT, 'modelcomparison.json')
histories, val_accuracies, val_losses = mcfly.find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val, y_val_binary,
                                                                           models, nr_epochs=5,
                                                                           subset_size=2000,
                                                                           verbose=True,
                                                                           outputfile=outputfile)
print('Details of the training process were stored in ',outputfile)

Training model 0 CNN
Instructions for updating:
Use tf.cast instead.
Train on 2000 samples, validate on 4983 samples
Epoch 1/5
2000/2000 [==============================] - 24s 12ms/step - loss: 155.9839 - acc: 0.3270 - val_loss: 121.4596 - val_acc: 0.4164
Epoch 2/5
2000/2000 [==============================] - 15s 8ms/step - loss: 96.8957 - acc: 0.3565 - val_loss: 76.0037 - val_acc: 0.3117
Epoch 3/5
2000/2000 [==============================] - 16s 8ms/step - loss: 61.4584 - acc: 0.4140 - val_loss: 49.5331 - val_acc: 0.2806
Epoch 4/5
2000/2000 [==============================] - 16s 8ms/step - loss: 41.1918 - acc: 0.4330 - val_loss: 34.4188 - val_acc: 0.3761
Epoch 5/5
2000/2000 [==============================] - 15s 8ms/step - loss: 29.5706 - acc: 0.4465 - val_loss: 25.6414 - val_acc: 0.3662
Training model 1 DeepConvLSTM
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2000 samples, validate on 4983 samples
Epoch 1/5
2000/2000 [=======================

In [14]:
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons.to_csv(os.path.join(PATH_OUTPUT, 'modelcomparisons.csv'))

modelcomparisons

,model,train_acc,train_loss,val_acc,val_loss
0,"{'learning_rate': 0.0001353096649122913, 'regu...",0.4465,29.570633,0.366245,25.641414
1,"{'learning_rate': 0.06640809560572995, 'regula...",0.4420,1.376897,0.480835,1.345288
2,"{'learning_rate': 0.0009021310490093333, 'regu...",0.4575,1.520157,0.480835,1.453386
3,"{'learning_rate': 0.0011361429425767724, 'regu...",0.4870,1.353813,0.480835,1.347448
4,"{'learning_rate': 0.00036451786911403326, 'reg...",0.4705,2.216500,0.304636,4.128861
5,"{'learning_rate': 0.061905645467897834, 'regul...",0.4205,1.552352,0.480835,1.410927
6,"{'learning_rate': 0.009962109969681159, 'regul...",0.4825,1.332951,0.480835,1.338628
7,"{'learning_rate': 0.013922563533153916, 'regul...",0.4815,1.975973,0.480835,1.728680


## Problem occuring here:
+ Input values are TOO low --> add normalization
+ Most labels are in "6" (group 2, stimuli 3). This are about 48.8%. So here the networks mostly just learned that it's OK to just always output "6" as a prediction...